# Ejercicio 2 Prácticas LC. Curso 2020-2021
### Jaime Ferrando Huertas

In [1]:
#IMPORTS
import nltk
from nltk.corpus import cess_esp
from nltk.tag import hmm
from nltk.tag import tnt
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import math

nltk.download('cess_esp')

[nltk_data] Downloading package cess_esp to /Users/jaime/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


True

# Ejercicio 1

In [2]:
#Cargamos el corpus

corpus_sentences = cess_esp.tagged_sents()
number_sentences = len(corpus_sentences)
print("number of sentnces:", number_sentences)
print("number of words:", number_sentences)


number of sentnces: 6030
number of words: 6030


In [3]:
#Procesamos el corpus
processed_corpus = []
for sentence in corpus:
    tmp_sentence = []
    for word in sentence:
        if word[0] == "*0*":
            continue
        if word[1][0] == "v" or "F":
            new_word = (word[0], word[1][:3])
        else:
            new_word = (palabra_etiquetada[0], word[1][:2])
        tmp_sentence.append(new_word)
    processed_corpus.append(tmp_sentence)

NameError: name 'corpus' is not defined

In [ ]:
#Dividimos el corpus en 90% training 10% test

len_training = int(0.9*len(processed_corpus))
training_notcv = processed_corpus[:len_training]
test_notcv = processed_corpus[len_training:]

# Ejercicio 2

In [ ]:
#Entrenamos los etiquetadores morfosintacticos (modelos HMM Y TNT) en las particiones de entranimiento previamente hechas en el ejercicio 1 ya procesadas

#TNT
tagger_tnt = tnt.TnT()
tagger_tnt.train(training_notcv)
v=tagger_tnt.evaluate(test_notcv)
d = 1.96*math.sqrt((v*(1-v))/len(test_notcv))
ic = [round(v-d,3),round(v+d, 3)]
print("Precission for TNT without CV")
print(v)
print(f"Intervalo de confianza {ic}")

#HMM 
tagger_hmm = hmm.HiddenMarkovModelTagger.train(training_notcv)
v = tagger_hmm.evaluate(test_notcv)
d = 1.96*math.sqrt((v*(1-v))/len(test_notcv))
ic = [round(v-d,3),round(v+d, 3)]
print("Precission for HMM without CV")
print(v)
print(f"Intervalo de confianza {ic}")

El etiquetador basado en cadenas de markov nos da un valor de precision superior al etiquetador TNT

# Ejercico 3

In [ ]:
%%time
#Entrenamos los etiquetadores morfosintacticos (modelos HMM Y TNT) esta vez con el metodo de cross validation con 10-Fold, primero sin barajar el dataset y despues barajandolo

#Sin barajar
results_unshuffled_tnt = []
results_unshuffled_hmm = []
intervals_unshuffled_tnt = []
intervals_unshuffled_hmm = []
kf_unshuffled = KFold(n_splits=10, shuffle=False)
for train_index, test_index in kf_unshuffled.split(processed_corpus):
    #Split data
    training_cv = np.take(processed_corpus,train_index)
    test_cv = np.take(processed_corpus,test_index)
    #TNT
    tagger_tnt = tnt.TnT()
    tagger_tnt.train(training_cv)
    v=tagger_tnt.evaluate(test_cv)
    results_unshuffled_tnt.append(v)
    #Intervalos
    d = 1.96*math.sqrt((v*(1-v))/len(test_cv))
    ic = round(d,3)
    intervals_unshuffled_tnt.append(ic)
    #HMM
    tagger_hmm = hmm.HiddenMarkovModelTagger.train(training_cv)
    v = tagger_hmm.evaluate(test_cv)
    results_unshuffled_hmm.append(v)
    #Intervalos
    d = 1.96*math.sqrt((v*(1-v))/len(test_cv))
    ic = round(d,3)
    intervals_unshuffled_hmm.append(ic)


#Barajando
kf_shuffled = KFold(n_splits=10, shuffle=True)
results_shuffled_tnt = []
results_shuffled_hmm = []
intervals_shuffled_tnt = []
intervals_shuffled_hmm = []
for train_index, test_index in kf_shuffled.split(processed_corpus):
    #Split data
    training_cv = np.take(processed_corpus,train_index)
    test_cv = np.take(processed_corpus,test_index)
    #TNT
    tagger_tnt = tnt.TnT()
    tagger_tnt.train(training_cv)
    v=tagger_tnt.evaluate(test_cv)
    results_shuffled_tnt.append(v)
    #Intervalos
    d = 1.96*math.sqrt((v*(1-v))/len(test_cv))
    ic = round(d,3)
    intervals_shuffled_tnt.append(ic)
    #HMM
    tagger_hmm = hmm.HiddenMarkovModelTagger.train(training_cv)
    v = tagger_hmm.evaluate(test_cv)
    results_shuffled_hmm.append(v)
    #Intervalos
    d = 1.96*math.sqrt((v*(1-v))/len(test_cv))
    ic = round(d,3)
    intervals_shuffled_hmm.append(ic)



## Creamos tablas y comentamos los resultados

### Unshuffled

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#TNT

x=[i for i in range(10)]
y=results_unshuffled_tnt # results es una lista con los resultados de cada experimento
plt.axis([-1, 10, 0.80, 0.97])
plt.ylabel('Accuracy')
plt.xlabel('Fold')
plt.title('Cross validation 10 Folds TNT - Corpus unshuffled')
plt.plot(x,y,'ro')
Tic=intervals_unshuffled_tnt# Lista con los intervalos de confianza
plt.errorbar(x,y,yerr=Tic,linestyle='None')
plt.show()
plt.savefig('cv_tnt_unshuffled.png')

#HMM

x=[i for i in range(10)]
y=results_unshuffled_hmm # results es una lista con los resultados de cada experimento
plt.axis([-1, 10, 0.80, 0.97])
plt.ylabel('Accuracy')
plt.xlabel('Fold')
plt.title('Cross validation 10 Folds HMM - Corpus unshuffled')
plt.plot(x,y,'ro')
Tic=intervals_unshuffled_hmm# Lista con los intervalos de confianza
plt.errorbar(x,y,yerr=Tic,linestyle='None')
plt.show()
plt.savefig('cv_hmm_unshuffled.png')

Podemos observar como para nuestros dos etiquetadores (TNT y HMM) obtienen una mejor accuracy media con cross validation que en el primer ejercicio donde tenian valores peores (TNT: 0.82, HMM: 0.88). El haber evaluado con cross validation (sin barajar) tambien nos da la oportunidad de ver como se distribuye el error de nuestro etiquetador a lo largo del dataset, vemos como el ultimo 10% del dataset es el punto con peor accuracy para los dos etiquetadores y creemos que esto se debe a que contiene muestras no muy similares al 90% inicial de los datos y por lo tanto el etiquetador no puede aprender sobre este nuevo tipo de muestras.



### Shuffled

In [ ]:
#TNT
x=[i for i in range(10)]
y=results_shuffled_tnt # results es una lista con los resultados de cada experimento
plt.axis([-1, 10, 0.80, 0.97])
plt.ylabel('Accuracy')
plt.xlabel('Fold')
plt.title('Cross validation 10 Folds TNT - Corpus shuffled')
plt.plot(x,y,'ro')
Tic=intervals_shuffled_tnt# Lista con los intervalos de confianza
plt.errorbar(x,y,yerr=Tic,linestyle='None')
plt.show()
plt.savefig('cv_tnt_shuffled.png')

#HMM

x=[i for i in range(10)]
y=results_shuffled_hmm # results es una lista con los resultados de cada experimento
plt.axis([-1, 10, 0.80, 0.97])
plt.ylabel('Accuracy')
plt.xlabel('Fold')
plt.title('Cross validation 10 Folds HMM - Corpus shuffled')
plt.plot(x,y,'ro')
Tic=intervals_shuffled_hmm# Lista con los intervalos de confianza
plt.errorbar(x,y,yerr=Tic,linestyle='None')
plt.show()
plt.savefig('cv_hmm_shuffled.png')

Sin embargo si barajamos el dataset nos libramos de este inconveniente ya que teoricamente todas las muestras tienen una representacion equitativa en todos los Folds de cross validation, esto se refleja en las dos ultimas graficas donde vemos un error similar durante todos los folds de nuestro cross validation